<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/query_transformations/HyDEQueryTransformDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# HyDE 查询转换


如果您在Colab上打开这个笔记本，您可能需要安装LlamaIndex 🦙。


In [ ]:
!pip install llama-index

### 下载数据


In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

#### 加载文档，构建VectorStoreIndex


In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from IPython.display import Markdown, display

In [ ]:
# 加载文档documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

In [ ]:
index = VectorStoreIndex.from_documents(documents)

## 示例：HyDE改进了特定的时间查询


In [ ]:
query_str = "what did paul graham do after going to RISD"

#### 首先，我们进行*无*转换的查询：相同的查询字符串用于嵌入查找和总结。


In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

在去RISD之后，保罗·格雷厄姆继续追求他对绘画和艺术的热情。他在佛罗伦萨的贝利艺术学院绘画系上课，并参加了学校的入学考试。他还继续着手写他的著作《On Lisp》，并接受咨询工作来赚钱。在学校里，保罗·格雷厄姆和其他学生达成了一项协议，即教职员工不会要求学生学习任何东西，作为回报，学生也不会要求教职员工教授任何东西。保罗·格雷厄姆是为数不多的实际上给提供的裸体模特绘画的学生之一，而其他学生则把时间花在聊天或偶尔试图模仿他们在美国艺术杂志上看到的东西上。这位模特竟然就住在保罗·格雷厄姆家附近，她靠为当地古董商制作赝品和做模特谋生。


#### 现在，我们使用`HyDEQueryTransform`来生成一个假设的文档，并将其用于嵌入查找。


In [ ]:
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

在去RISD之后，保罗·格雷厄姆曾在Interleaf担任顾问，然后与罗伯特·莫里斯共同创立了Viaweb。他们开发了一款允许用户通过网络构建网站的软件，并从Idelle的丈夫朱利安那里获得了1万美元的种子资金。作为最初的法律工作和业务建议的回报，他们给了朱利安公司的10%股份。由于保罗·格雷厄姆欠税而净资产为负，因此种子资金对他来说是必不可少的。他们于1996年1月开业，拥有6家门店。

保罗·格雷厄姆之后在雅虎的期权到期后离开，回到了纽约。他恢复了以前的生活，但现在他变得富有了。他尝试绘画，但缺乏精力和雄心。最终，他搬回了剑桥，并开始致力于开发一个用于制作Web应用程序的Web应用程序。他招募了丹·吉芬和两名本科生来帮助他，但最终意识到自己并不想经营一家公司，决定将项目的一个子集作为开源项目进行开发。他和丹在他在剑桥购买的房子里致力于开发一种新的Lisp方言，他称之为Arc。他作为开源项目开发的子集是新的Lisp。


在这个例子中，`HyDE`显著提高了输出质量，通过准确地虚构了保罗·格雷厄姆在RISD毕业后的经历（见下文），从而提高了嵌入质量和最终输出。


In [ ]:
query_bundle = hyde(query_str)
hyde_doc = query_bundle.embedding_strs[0]

In [ ]:
hyde_doc

在1985年从罗德岛设计学院（RISD）毕业后，保罗·格雷厄姆开始了他的计算机编程职业生涯。他曾在多家公司担任软件开发人员，包括他在1995年共同创立的Viaweb公司。Viaweb最终在1998年被雅虎收购，而格雷厄姆利用收益成为了一名风险投资家。他于2005年创立了Y Combinator，这是一家初创企业加速器，已帮助推动了2000多家公司的发展，包括Dropbox、Airbnb和Reddit。格雷厄姆还撰写了几本关于编程和初创企业的书籍，他继续活跃于科技行业的投资领域。


## 失败案例1：在没有上下文的情况下，HyDE可能会误导查询的解释。


In [ ]:
query_str = "What is Bel?"

### 查询不经过转换会得到合理的答案


In [ ]:
response = query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

Bel是一种由Paul Graham在四年的时间里（2015年3月26日至2019年10月12日）用Arc编写的编程语言。它基于John McCarthy最初的Lisp，但添加了额外的功能。它是以代码形式表达的规范，旨在成为计算的形式模型，是图灵机的一种替代方案。


#### 使用`HyDEQueryTransform`查询结果是无意义的


In [ ]:
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)
response = hyde_query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

Bel是保罗·格雷厄姆的化名，他是这段背景信息的作者，当时需要种子资金来生活，并参与了一项成为Y Combinator模型的交易。


在这个例子中，`HyDE`在没有文档上下文的情况下错误地解释了Bel（见下文），导致了一个完全无关的嵌入字符串和较差的检索结果。


In [ ]:
query_bundle = hyde(query_str)
hyde_doc = query_bundle.embedding_strs[0]

In [ ]:
hyde_doc

> 贝尔是一个古老的闪米特神，起源于中东。他经常被与太阳联系在一起，有时被称为“天堂之主”。贝尔也被称为生育之神、丰饶之神和繁荣之神。他经常被描绘成一头公牛或一个带有公牛头的男人。在一些文化中，贝尔被视为创世神，负责创造宇宙。他还与地底世界联系在一起，有时被视为死亡之神。贝尔还与正义联系在一起，经常被视为无辜者的保护神。贝尔在包括犹太教、基督教和伊斯兰教在内的许多宗教中都是一个重要的人物。


## 失败案例2：HyDE可能会对开放式查询产生偏见


In [ ]:
query_str = "What would the author say about art vs. engineering?"

#### 查询不经过转换会得到一个合理的答案


In [ ]:
response = query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

作者可能会说，艺术和工程是两种需要不同技能和方法的学科。艺术更注重表达和创造力，而工程更注重解决问题和技术知识。作者还建议，艺术学校并不总是提供与工程学校相同水平的严谨性，绘画学生通常被鼓励发展个人风格，而不是学习绘画的基础知识。此外，作者可能会指出，工程可以提供比艺术更多的财务稳定性，正如作者自己需要种子资金来维持生活，同时推动公司的发展所证明的那样。


#### 使用`HyDEQueryTransform` 进行查询会导致更加偏向的输出


In [ ]:
response = hyde_query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

作者可能会说，艺术比工程更持久和独立。他们提到，今天编写的软件在几十年后就会过时，系统工程也不会持久。相比之下，他们指出绘画可以保存数百年，而且有可能以艺术家的身份谋生。他们还提到，作为艺术家，你可以真正独立，不需要老板或研究资金。此外，他们指出艺术可以成为那些无法获得传统就业机会的人的收入来源，比如例子中的模特儿能够通过为当地古董商模特和制作赝品而谋生。
